# Project Cars4you (Group 5)

### Import & load Data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import seaborn as sns

df_cars_train = pd.read_csv("train.csv")
df_cars_test = pd.read_csv("test.csv")

### Explorative Data Analysis (EDA)

TASK I (3 Points): Descriptive Statistics, Inconsistency Check, Visual Data Explorance, Extraction of Relevant Insights, Multivariate Relationships  => Explain

In [0]:
# Overview of structure and data types
df_cars_train.info()

# Check for duplicates
print(f"\nDuplicate rows: {df_cars_train.duplicated().sum()}") # 0

# Check for duplicate carID
print(f"\nDuplicate carID: {df_cars_train['carID'].duplicated().sum()}") # 0

# Check for null values in column CarID
print(f"\nNull values in column carID: {df_cars_train['carID'].isnull().sum()}") # 0

# Findings:
#   - missing values in every column except carID
#   - year, mpg, previousOwners, hasDamage as float seems weird
#   - no duplicates in all columns, no null values in carID

In [0]:
# print exact unique values of df_cars_train before doing describe
for col in df_cars_train.columns:
    print(col, df_cars_train[col].unique())

# describe all
df_cars_train.describe(include="all").T

In [0]:
# Univariate Exploration of relevant Columns

sns.set(style="whitegrid")

num_cols = ['price','mileage','tax','mpg','engineSize','paintQuality%','previousOwners','year']

# colors
hist_color = '#1f77b4'   # dark blue
box_color = '#ff7f0e'    # warm orange

# figure: 2 features per row (4 plots per row = hist + boxplot per feature)
n_features = len(num_cols)
n_rows = int(np.ceil(n_features / 2))
fig, axes = plt.subplots(n_rows, 4, figsize=(16, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    # histogram
    sns.histplot(df_cars_train[col], bins=25, color=hist_color, kde=False, ax=axes[i*2])
    axes[i*2].set_xlabel('')
    axes[i*2].set_ylabel('')
    axes[i*2].set_title(f'{col}', fontsize=11, pad=12)

    # boxplot (vertical)
    sns.boxplot(y=df_cars_train[col], ax=axes[i*2 + 1], color=box_color)
    axes[i*2 + 1].set_xlabel('')
    axes[i*2 + 1].set_ylabel('')
    axes[i*2 + 1].set_title(f'{col}', fontsize=11, pad=12)

# hide any unused axes
for j in range(i*2 + 2, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('univariate distributions of numerical features', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

%md
##### Findings after Descriptive Statistics and Inconsistency Check

carID:  
- sequential numeric identifier, ranges from **0–75,972**  
- no duplicates expected, used only as index/key  

brand:  
- **72 unique brands** with severe spelling and capitalization inconsistencies (**Ford**, **ford**, **FOR**, **ord**, **For**, etc.)  
- contains partial or truncated names (**w**, **MW**, **Ope**, **Mercede**) → heavy cleaning required  
- **Ford dominates (~15k entries)**, followed by **BMW**, **VW**, **Mercedes**, **Toyota**, **Audi**, **Skoda**, **Hyundai**  
- strong class imbalance → use frequency encoding or median price per brand later  

model:  
- **735 unique entries** with inconsistent formatting, spacing, capitalization, and partial strings  
- duplicates of same model under variations (e.g. *“focus”*, *“ FOCUS”*, *“ Focu”*, *“Focus”*)  
- many small typos, truncated or malformed entries (*“Focu”*, *“EcoSpor”*, *“Gol”*, *“Yeti Outdoo”*, etc.)  
- heavy normalization needed → strip whitespace, lowercase, and fix common truncations  
- dominated by popular models like **Ford Focus**, **VW Golf**, **Vauxhall Astra/Corsa**, **Skoda Octavia**  

year:  
- values range from **1970–2024**, mean ≈ **2017**  
- years after **2020** are unrealistic (future registration) → invalid entries  
- decimals in year (e.g. **2023.367**) → data corruption, round to nearest int  
- older outliers before **2000** rare, likely classic or miscoded entries  
- can derive **age = 2020 - year** for modeling  

transmission:  
- **40 distinct entries**, mostly spelling variants of *manual*, *automatic*, *semi-auto*  
- common corruptions: *manua*, *anual*, *semi-aut*, *utomatic*, *nknow*, etc.  
- some leading/trailing spaces (*' manual '*, *' Manual '*)  
- categories should be reduced to clean labels: **manual**, **automatic**, **semi-auto**, **unknown**  

mileage:  
- range **–58,540 → 323,000**, mean ≈ **23k** → negatives invalid  
- missing values around **1.5k**  
- strong right skew, some extreme outliers >**250k miles**  
- negative or zero values should be filtered or replaced with abs()  

tax:  
- range **–91 → 580**, mean ≈ **120** → invalid negatives present  
- normal values cluster around **125–145**  
- decimals and small negatives appear due to calculation/entry errors  
- likely strong right skew → a few cars taxed over **500**  
- needs capping and replacement for negatives  

mpg:  
- range **–43 → 470**, mean ≈ **55** → negative and extreme outliers exist  
- typical real range **30–70 mpg**, but some values like **470** unrealistic  
- invalid entries indicate unit mix-up or input noise  
- expected inverse relation with **engine size** and **price**  

engineSize:  
- range **–0.1 → 6.6L**, mean ≈ **1.66L**, std ≈ **0.57**  
- several decimals and negative/zero values → invalid  
- expected valid range **0.6–6.0L**  
- most cars between **1.2–2.0L** → compact to mid-size engines  
- positive correlation with **price** and **tax**  

paintQuality%:  
- range **1.6 → 125.6%**, mean ≈ **64.6%**  
- values above **100%** unrealistic → scaling error  
- some extremely low values (**≈1–3%**) indicate outliers or noise  
- most cars between **50–80%** → average paint quality  

previousOwners:  
- range **–2.3 → 6.26**, mean ≈ **2**  
- negative values invalid → likely placeholder or encoding issue  
- most between **0–3** → typical secondhand ownership distribution  
- outliers >**6** likely data entry errors or mis-scaling  
- likely negative correlation with **price**  

hasDamage:  
- only values are **0 and NaN** → no variation  
- unclear if NaN means damaged → convert to int and verify meaning  
- likely nonfunctional feature → **drop**  

price (target):  
- range **£450–£159,999**, mean ≈ **£16.9k**, median ≈ **£14.7k** → right-skewed  
- typical cars priced **£10k–£21k**, few luxury outliers inflate mean  
- consistent integer values, no missing or obvious anomalies  
- strong dependence expected on **mileage**, **year**, **engine size**, and **brand**
</div>


In [0]:
# pearson and spearman correlation

num_cols = ['price','mileage','tax','mpg','engineSize','paintQuality%','previousOwners','year']

corr_pearson = df_cars_train[num_cols].corr(method='pearson', numeric_only=True).round(2)
corr_spearman = df_cars_train[num_cols].corr(method='spearman', numeric_only=True).round(2)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
mask = np.triu(np.ones_like(corr_spearman, dtype=bool))

sns.heatmap(corr_pearson, mask=mask, annot=True, cmap='coolwarm', center=0, fmt='.2f',
            linewidths=0.5, ax=axes[0], cbar=False)
axes[0].set_title('pearson correlation (linear)')

sns.heatmap(corr_spearman, mask=mask, annot=True, cmap='coolwarm', center=0, fmt='.2f',
            linewidths=0.5, ax=axes[1], cbar_kws={'label': 'correlation strength'})
axes[1].set_title('spearman correlation (monotonic)')

plt.suptitle('comparison of pearson vs spearman correlation', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()


##### Why we decided to use Spearman correlation:

- the numeric variables (price, mileage, mpg, tax, engineSize) are **non-normally distributed and contain outliers**, causing linear measures like pearson to distort correlation strength.  
- **spearman** evaluates *monotonic* relationships based on rank order rather than exact linearity, making it **robust to skewness and outliers**.  
- this allows us to correctly capture the direction and magnitude of real-world trends, such as price decreasing with mileage, even if not perfectly linear.  
- after cleaning and scaling the data, we will revisit **pearson correlation** for linear modeling checks, but **spearman is more appropriate for initial EDA**.

##### Findings after correlation:

- price shows **strong positive correlation with year (0.60)** → newer cars are priced higher  
- price is **positively correlated with engineSize (0.56)** → larger engines increase car value  
- price is **moderately negatively correlated with mileage (–0.51)** → more driven cars lose value  
- mpg correlates **negatively with price (–0.39)** → efficient cars are typically smaller and cheaper  
- tax has a **moderate positive correlation (0.31)** with price → more expensive cars often have higher taxes  
- year and mileage have a **very strong negative correlation (–0.78)** → newer cars have lower mileage  
- mpg and tax are **strongly negatively correlated (–0.55)** → efficient cars usually taxed less  
- engineSize and mpg **correlate negatively (–0.20) → larger engines are less fuel-efficient  
- paintQuality% and previousOwners show **near-zero correlations** with all other variables → low predictive relevance  
- overall, **price mainly depends on year, engineSize, mileage, and mpg**, which align with intuitive market behavior


In [0]:
# Multivariate Analysis



%md
##### Findings after Multivariate Analysis
